# Daily Reminder Dataset

In [1]:
import pandas as pd
import os

file_path = "D:\Machine_Learning_Project\AI_for_Elderly_Care_and_Support\Dataset\daily_reminder.csv"
daily_reminder_df = pd.read_csv(file_path)


<>:4: SyntaxWarning: invalid escape sequence '\M'
<>:4: SyntaxWarning: invalid escape sequence '\M'
C:\Users\DELL\AppData\Local\Temp\ipykernel_1568\1909976674.py:4: SyntaxWarning: invalid escape sequence '\M'
  file_path = "D:\Machine_Learning_Project\AI_for_Elderly_Care_and_Support\Dataset\daily_reminder.csv"


In [2]:
# Displaying basic info()
daily_reminder_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Device-ID/User-ID       10000 non-null  object 
 1   Timestamp               10000 non-null  object 
 2   Reminder Type           10000 non-null  object 
 3   Scheduled Time          10000 non-null  object 
 4   Reminder Sent (Yes/No)  10000 non-null  object 
 5   Acknowledged (Yes/No)   10000 non-null  object 
 6   Unnamed: 6              0 non-null      float64
dtypes: float64(1), object(6)
memory usage: 547.0+ KB


In [3]:
daily_reminder_df.head()

,Device-ID/User-ID,Timestamp,Reminder Type,Scheduled Time,Reminder Sent (Yes/No),Acknowledged (Yes/No),Unnamed: 6
0,D1000,1/2/2025 11:25,Exercise,13:00:00,No,No,NaN
1,D1001,1/3/2025 2:52,Hydration,13:00:00,Yes,Yes,NaN
2,D1002,1/8/2025 13:50,Appointment,13:30:00,No,No,NaN
3,D1003,1/5/2025 5:16,Exercise,8:00:00,No,No,NaN
4,D1004,1/1/2025 4:20,Medication,11:30:00,No,No,NaN


In [4]:
# Droping the empty column
daily_reminder_df = daily_reminder_df.drop(columns=["Unnamed: 6"])

In [5]:
# REnaming the column

daily_reminder_df = daily_reminder_df.rename(columns={
    "Device-ID/User-ID": "user_id",
    "Timestamp": "timestamp",
    "Reminder Type": "reminder_type",
    "Scheduled Time": "schedule_time",
    "Reminder Sent (Yes/No)": "reminder_sent",
    "Acknowledged (Yes/No)": "acknowledged"
})


In [6]:
# Converting the timestamp to datetime format
daily_reminder_df["timestamp"] = pd.to_datetime(daily_reminder_df["timestamp"])

In [7]:
# Converting schedule time to time format

daily_reminder_df["schedule_time"] = pd.to_datetime(daily_reminder_df["schedule_time"], format='%H:%M:%S').dt.time

In [8]:
# Converting Yes/No columns to binary (1= yes, 0 = No)
daily_reminder_df["reminder_sent"] = daily_reminder_df["reminder_sent"].map({"Yes": 1, "No": 0})
daily_reminder_df["acknowledged"] = daily_reminder_df["acknowledged"].map({"Yes": 1, "No": 0})

In [9]:
# Final code
daily_reminder_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   user_id        10000 non-null  object        
 1   timestamp      10000 non-null  datetime64[ns]
 2   reminder_type  10000 non-null  object        
 3   schedule_time  10000 non-null  object        
 4   reminder_sent  10000 non-null  int64         
 5   acknowledged   10000 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 468.9+ KB


In [10]:
daily_reminder_df.head()

,user_id,timestamp,reminder_type,schedule_time,reminder_sent,acknowledged
0,D1000,2025-01-02 11:25:00,Exercise,13:00:00,0,0
1,D1001,2025-01-03 02:52:00,Hydration,13:00:00,1,1
2,D1002,2025-01-08 13:50:00,Appointment,13:30:00,0,0
3,D1003,2025-01-05 05:16:00,Exercise,08:00:00,0,0
4,D1004,2025-01-01 04:20:00,Medication,11:30:00,0,0


In [11]:
# Saving the cleaned data to cleaned data folder
data_folder = "../cleaned_data"
file_name = "cleaned_daily_reminder_dataset.csv"
file_path = os.path.join(data_folder, file_name)
daily_reminder_df.to_csv(file_path, index=False)

## Inserting the daily reminder into PostgreSQL

In [12]:
import psycopg2

# sql connection details
conn = psycopg2.connect(
    dbname = "elderly_care",
    user = "postgres",
    password = "system",
    host = "localhost",
    port = "5432"
    
)
cur = conn.cursor()

In [13]:
# Inserting data
for _, row in daily_reminder_df.iterrows():
    cur.execute("""
        INSERT INTO daily_reminder (
            user_id, timestamp, reminder_type,
            schedule_time, reminder_sent, acknowledged
        )
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['user_id'], row['timestamp'], row['reminder_type'], str(row['schedule_time']), row['reminder_sent'], row['acknowledged']))

In [14]:
# Commit and close
conn.commit()
cur.close()
conn.close()
print(" Data inserted successfully!")

 Data inserted successfully!
